###Web of Science API
This code is designed to interact with the WOS API using "premium" access protocols.

-    `wos.py` contains code for interacting with the API directly.
-    `woscalls.py` includes calls made to the `WOS` class
-    `metawos.py` can be used to extract metadata from search results.